In [ ]:
from mstrio.modeling import *
from mstrio.connection import Connection

import inspect
import getpass

In [ ]:
BASE_URL = "http://10.23.35.137:8080/MicroStrategyLibrary/api"  # Insert URL for your env here
MSTR_USERNAME = "administrator"  # Insert your env username here
MSTR_PASSWORD = getpass.getpass()  # insert your mstr password here
PROJECT_ID = "2FAF47F94024E5EE35048789DCF28FB8"  # Insert you project ID here

In [ ]:
conn = Connection(BASE_URL, MSTR_USERNAME, MSTR_PASSWORD, project_id=PROJECT_ID, login_mode=1)

# Discover Metrics

## List

In [ ]:
print(list_metrics.__doc__)

In [ ]:
inspect.signature(list_metrics)

In [ ]:
metrics = list_metrics(conn, project_id="2FAF47F94024E5EE35048789DCF28FB8")
metrics

In [ ]:
list_metrics(conn, search_pattern=SearchPattern.EXACTLY, name="Revenue")

## Get by id

In [ ]:
print(Metric.__init__.__doc__)

In [ ]:
metric = Metric(conn, id="4C05177011D3E877C000B3B2D86C964F")

## Properties

In [ ]:
metric.list_properties()

In [ ]:
metric.metric_subtotals

In [ ]:
metric.id

In [ ]:
metric.name

In [ ]:
metric.sub_type

In [ ]:
print(metric.sub_type.__doc__)

In [ ]:
metric.aggregate_from_base

In [ ]:
metric.dimensionality

# Create Metric

## Create Metric Expression

In [ ]:
expression_data = {
        "tokens": [
          {
            "value": "Sum([Revenue])"
          }
        ]
      }

In [ ]:
def create_expression_with_tokens(expression):
    tokens_data = expression["tokens"]
    tokens = []
    for t in tokens_data:
        # Simple Token with just value
        tokens.append(Token(value=t["value"]))
    # This doesn't work with simple token
    return Expression(
        tokens=tokens
    )

In [ ]:
expression_obj = create_expression_with_tokens(expression_data)

## Helper function

In [ ]:
def get_enum_from_str(type, s):
    for t in type:
        if t.value == s:
            return t
    raise Exception("Incorrect string "+s +" for type: " + type)


## Create Metric Dimty

In [ ]:
inspect.signature(Dimensionality.__init__)

In [ ]:
dimty_data = {
        "dimtyUnits": [
          {
            "dimtyUnitType": "report_base_level",
            "aggregation": "normal",
            "filtering": "apply",
            "groupBy": True
          }
        ],
        "excludeAttribute": False,
        "allowAddingUnit": True
      }

In [ ]:
def create_dimty(dimty_data):

    if dimty_data is None: return None

    dimty_units = dimty_data["dimtyUnits"]
    du_objects = []

    for du in dimty_units:
        type = get_enum_from_str(DimensionalityUnit.DimensionalityUnitType, du["dimtyUnitType"])
        aggregation = get_enum_from_str(DimensionalityUnit.Aggregation, du["aggregation"])
        filtering = get_enum_from_str(DimensionalityUnit.Filtering, du["filtering"])
        group_by = du["groupBy"]
        du_objects.append(DimensionalityUnit(
                dimensionality_unit_type=type,
                aggregation=aggregation,
                filtering=filtering,
                group_by=group_by,
                relative_position=None,
                axis_collection=None
            ))
    return Dimensionality(dimensionality_units=du_objects,
                          exclude_attribute=dimty_data.get("excludeAttribute"),
                          allow_adding_unit=dimty_data.get("allowAddingUnit"))
    

In [ ]:
dimty_obj = create_dimty(dimty_data)

## Create Metric

In [ ]:
print(Metric.create.__doc__)

In [ ]:
METRIC_NAME = "Revenue"
METRIC_DESC = "Renenue"
METRIC_SUBTYPE="metric"
DESTINATIN_FOLDER_ID = "95C3B713318B43D490EE789BE27D298C"


In [ ]:
metric_obj = Metric.create(conn,
                           name=METRIC_NAME,
                           description=METRIC_DESC,
                           sub_type=get_enum_from_str(ObjectSubType, "metric"),
                           destination_folder=DESTINATIN_FOLDER_ID,
                           expression=expression_obj,
                           dimensionality=dimty_obj)
                           
metric_obj